# Generating translation maps for different grading systems

In this exercise we are going to write code to automatically generate a translation from a grading scheme `old_scheme` to another one called `new_scheme`.
Our code is going to be general, so by the end we will have a solution that works for any given two schemes and only makes very few assumptions.
Additionally, we are going to implement different strategies to generate the translation.


Consider the grading system below:

<img src='img/grading.png'/>

For convenience let's encode each grading scheme in the following way.

```python
old_scheme = [('A+', 91), ('A', 75), ('B', 60), ('C', 50),
              ('D', 35), ('E', 20), ('U', 0)]
              
new_scheme = [(1, 90), (2, 85), (3, 80), (4, 75), 
              (5, 65), (6, 45), (7, 20), (8, 0)]
```

Each scheme is a list of tuples where the first item of the tuple corresponds to the grade and the second corresponds to the lower bound of the corresponding range.

We are asked to compute a function `old2new('A', scheme1, scheme2, assignment_func)` that computes the grade according to grading scheme `scheme2` corresponding
to the input grade 'A' corresponding to grading scheme `scheme1`.

The first thing to notice is that this function can be thought as just the application of another function that generates a translation table.
Given a translation table like:

```python
table = {'A+': 1, 'A': 4, 'B': 6, 'C': 6, 'D': 7, 'E': 7, 'U': 8}
```

our we can translate by just accessing items in the dictionary `table['A+'] >>> 1`.

The problem we are facing is that for some entries in the old scheme there are many possible translations, and yet we are asked to return one.
In general, there will be different ways of deciding which grade in 

In order to make that decision we need to consider which are possible candidates of a given input grade in the old scheme.
For instance, in the picture above, for grade 'A' we only have to consider entries 1, 2, 3 and 4 of the new scheme.
In order to make an informed decision, it is going to be useful to know the degree of overlap of each grade in the new scheme
with the target grade in the old scheme.
For instance, for our example, 'A' overlaps with 1 only by 1 point (90), but with 2 by 4 points for each of 2 (85-89), 3 (80-84) and 4 (75-79).

So the first step is to write a function `span(lower1, upper1, lower2, upper2)` that output the overlapping span.

```python
span(75, 90, 85, 89)
4
```

We can now use `span` to compute the set of candidate grades in the new scheme for a given grade in the old scheme.
This is done with `get_candidates(target, scheme)`, where target is a tuple of (lower bound, upper bound) for the grade in the old scheme.
The output of `get_candidates` is a list of tuples, where each tuple is a grade in the new scheme and its span with the target grade.

For instance, for the range corresponding to 'A' (75, 90), we get the following output.

```python
get_candidates((75, 91), new_scheme) # 91 since the upper bound is not inclusive
>>> [(1, 1), (2, 4), (3, 4), (4, 4)]
```

So, having all the ingredients, let's now think about how to generate the translation table. 
As usually, we are going to abstract this computation in a new function. Let's call it `generate_alignment`.

`generate_alignment` takes two schemes and an assignment function as input and returns a translation table. 

```python
generate_alignment(assignment, scheme1, scheme2)
{'A+': 1, 'A': 4, 'B': 6, 'C': 6, 'D': 7, 'E': 7, 'U': 8}
```

(Notice how functions can also be passed as arguments to another functions (this is key to building big and complex coding projects).)

The key for this function is actually this other function called `assignment` that implements a particular translation strategy.
Each assignment function has to have the same signature and should return the same output type. Otherwise we wont be able to swap
assignments. The input to an assignment function is a target (tuple of lower and upper bound) and the candidate set.
It's up to you to implement different strategies (think about the median grade in the set of candidates. Another possibility is
to pick the largest candidate span). Here we show you how a random assignment would work:

```python
def random_assignment(target, candidates):
    import random
    idx = random.randint(0, len(candidates)-1) # randint takes an inclusive upper bound
    grade, _ = candidates[idx]
    
    return grade
```

Once you are done, try your generated translations with different schemes. For instance, try to translate from the american grading scheme
to the german one. You can also generate translations in inversed order and you don't have to change your code a single bit!